# Cluster Analysis with K-means (MNIST Handwritten Digits)

## About Clustering
#### Clustering is an unsupervised learning problem where we group data according to some similarity hidden within the data. Typically in clustering we have data without labels where the data is represented as a matrix $X \in \mathbb{R}^{NxD}$ over $N$ observations and $D$ dimensions or features. K-means is a clustering or unsupervised learning method where we want to break $N$ observations into $K$ groups. This form of clustering is also known as Flat Clustering.

#### In K-means we want to select groups where . . .

#### 1) High intercluster similarity: $\sum_{k}\sum_{n \neq m\in k} d(x_n,x_m)$ should be low

#### 2) Low intracluster simularity: $\sum_{k \neq k'}\sum_{n \in k}\sum_{m \in k'} d(x_n,x_m)$ should be high

#### where $d(x,mu) = \Bigl(\sqrt{\sum_{k=1}^n\mid(x_k - mu_k)\mid^2}\Bigr)^2$


#### K-means operates in high-diemnsional space  $\mathbb{R}^{NxD}$ so we need a method to measure distance between two points in this space ($Euclidian \space space$) known as the $Euclidian \space norm$ or $L^2 norm$. Mathematically, a norm is a total size or length of all vectors in a vector space or matrices.  The $L^2 norm$ is used as a standard quantity for measuring a vector difference. We will see this later. In  $X \in \mathbb{R}^{2}$ it looks something like this.


![title](Euclidian.png)


#### The idea behind the K-means algorithm is to propose a defined number of clusters $K$ and then iteratively fix the errors to optimize the criteria discussed above. 

####            $\bullet$ Initialize cluster centers
####            $\bullet$ Assign each data point to the closeest center
####            $\bullet$ Recompute the centers as the means of the assigned data points
####            $\bullet$ If the assignments have changed, go to (2)


#### This is the k-means algorithm. It is very sensative to initialization, but converges quickly.
#### There are a couple different ways to initialize the cluster centers also called the centroids. 

####            $\bullet$ Random points drawn from a "reasonable" input space. What's reasonable?
####            $\bullet$ First select a random data point, then a second data point as far away as possible, then a third data point as far away as possible, and so on, and so on....



## Environment Setup

#### We will try and cluster some handwritten digits from the MINST database.

#### First we will import some Python packages that we will use.

In [ ]:
import math
import numpy as np
import scipy.io as sio
import numpy.matlib as ml
from scipy import ndimage
from itertools import repeat
from numpy import linalg as la
%matplotlib inline 
import matplotlib.pyplot as plt

## Define the K-means Algorithm

#### We will first need a function to calcualte the minimum distance between a set of vector features. See if you can figure out what is going on here .... Do you see where the $L^2 norm$ is being calculated?

In [ ]:
def minDistance(X, C):
    len, dim = C.shape
    D = ml.zeros((len, 1), 'float64')
    for i in range(0, len):
        D[i] = la.norm(X - C[i,:],2)**2
    k = D.argmin()
    dist = min(D)[0,0]
    return k, dist

#### Next we will define the k-means algorithm that we described above. Note that we have implemented both random initialization and furthest centroid distance initialization. Read the comments in the code to try and figure out what is going on.

In [ ]:
def kmeans(X, K, init):
    N, D  = X.shape
    if (K >= N):
        raise Exception('kmeans: you are trying to make too many clusters!');
    if (init == 'random'):
        # initialize by choosing K (distinct!) random points: we do this by
        # randomly permuting the examples and then selecting the first K points
        perm = np.random.permutation(N);
        perm = perm[:K]
        # the centers are given by selected vectors
        mu = X[perm, :]
        # we leave the assignments unspecified -- they'll be computed in the
        # first iteration of K means
        z = ml.zeros((N, 1), 'float64')
    elif (init == 'furthest'):
        # Randomly choose the first center c_1.  
        # then, let c_2 = argmax dist(x_i, c_1).
        # let  c_3 = argmax dist [ dist(x_i, c_1), dist(x_i, c_2) ] 
        # and so on....  
        perm = np.random.permutation(N);
        mu = X[perm[0], :].reshape(1, D)
        XT = np.delete(X, perm[0], 0)
        for i in range(1,K):
            len, dim = XT.shape
            DX = ml.zeros((len,1), 'float64')
            for j in range(0, len):
                k, dist = minDistance(XT[j,:].reshape(1, dim), mu);
                DX[j] = dist;
            indc = DX.argmax()
            r, c = mu.shape
            mu = np.append(mu, XT[indc,:].reshape(1, dim))
            mu = mu.reshape((r + 1), c)
            XT = np.delete(XT, indc, 0)          
        # again, don't bother initializing z
        z = ml.zeros((N, 1), 'float64')
    else:
        raise Exception('unknown initialization: use "furthest" or "random"');
    # begin the iterations.  we'll run for a maximum of 20, even though we
    # know that things will *eventually* converge.
    for itr in range(0, 20):
        # in the first step, we do assignments: each point is assigned to the
        # closest center.  we'll judge convergence based on these assignments,
        # so we want to keep track of the previous assignment
        oldz = np.array(z);
        for n in range(0,N):
            # assign point n to the closest center
            k, dist = minDistance(X[n,:].reshape(1, D), mu);
            z[n] = k;  
            # check to see if we've converged
        if all(oldz==z):
            break;
        # re-estimate the means            
        for k in range(0,K):
            mu[k,:] = X[np.where(z == k)[0], :].mean(0).reshape(1, D)             
        # final: compute the score
    score = 0;
    for n in range(0, N):
        # compute the distance between X(n,:) and it's associated mean
        score = score + la.norm(X[n,:] - mu[int(z[n][0,0]),:])**2;
    return (mu, z, score)


## Reading in our Dataset

#### Let's read in our MNIST handwritten digit images. We are going to cheat a little, we are borrowing this data set from MATLAB which gives us pre-populated matricies to work with.

In [ ]:
mnist = sio.loadmat('mnist.mat')
testX = mnist['testX']
testY = mnist['testY']
trainX = mnist['trainX']
trainY = mnist['trainY']

## Defining Clusters

#### We will analyze our handwritten digits by evaluating diffent cluster sets. We will look at $K \in \{2, 5, 10, 15, 20\}$. 

#### 1) What do you think the score is being produced? $You{\space}should{\space}know.$ 
#### 2) Is a higher score or lower score better?

In [ ]:
X = trainX
Y = trainY
allK = [2, 5, 10, 15, 20]
scores = list(repeat(0, len(allK)))
zs = []
mus = []

for ii in range(0, len(allK)):
    print("K = %d..." % allK[ii], end="", flush=True)
    bestScore = float("inf");
    bestZ     = 0;
    bestMu    = 0;
    for rep in range(1,3):
        mu,z,s = kmeans(X, allK[ii],'furthest');
        if (s < bestScore):
            bestScore = s;
            bestZ = z;
            bestMu = mu;
    print(" --> score %g\n" % bestScore) 
    scores[ii] = bestScore
    zs.append(bestZ)
    mus.append(bestMu)
  


## Statistical Metrics for Evaluation

#### We need to calculate some statistical measures to evaluate our clusters. We will display a graph of the regularlzed score with $AIC$ and $BIC$. All of these scoures are normalized so that they take a maximum value of 1 so they fit on the same plot. 

#### 1) What does $BIC$ and $AIC$ tell us? Can you figure out if a higher or lower score is better? 
#### 2) What are the trends of the plots as K increases? 
#### 3) Based on each score, BIC and AIC, what value of K would you choose to use?


In [ ]:
bic = list(repeat(0, len(allK)))
aic = list(repeat(0, len(allK)))
for ii in range(0, len(allK)):
    N = len(Y)
    bic[ii] = N * np.log(scores[ii] / N) + allK[ii] * np.log(N);
    aic[ii] = N * (np.log(2*math.pi*scores[ii] / N) + 1) + 2 * allK[ii];
bic = bic / max(bic)
aic = aic / max(aic)
scores = list(scores / max(scores))   

## Graphing the Scores

#### Can you answer the questions above?

In [ ]:
plt.figure("Figure 1")
plt.title('K versus score for digits data')
plt.plot(allK, scores, linestyle='-', marker='o', color='b')
plt.plot(allK, aic, linestyle='-', marker='x', color='g')
plt.plot(allK, bic, linestyle='-', marker='s', color='r')
plt.ylim(ymax = 1.0, ymin = 0.6)
plt.legend(['normalized score', 'normalized BIC', 'normalized AIC'], loc='lower left')
plt.ylabel('score')
plt.xlabel('K')
plt.show()

## Visualizing the Cluster Centroids

#### Here is some code that will allow us to visualize the cluster centroids. 


In [ ]:
def draw_digits(X,Y):
    N, D = X.shape
    DY = math.floor(math.sqrt(N));
    DX = math.ceil(N / DY);
    for n in range(0, N):
        plt.suptitle("K = " + str(N) + " Clusters")
        plt.subplot(DY, DX, n+1)
        Z = ml.uint8(255*X[n,:].reshape(28,28).T)
        im = plt.imshow(ndimage.rotate(Z, 90), interpolation='bilinear', origin='lower')
        plt.axis('off')    
        plt.title('C = ' + str(Y[n]))


## Data Visualization of our Handwritten Digit Clusters

#### 1) Take a look, what do you think -  do the clusters seem reasonable? 
#### 2) For each of the figures, how many "real" digits can you recognize?  Which ones are missing? Which ones seem to have been merged?
#### 3) What seems to be the best number of clusters considering all the graphs?

In [ ]:
plt.figure("Figure 2")
draw_digits(mus[0], list(range(1, allK[0]+1)))
plt.figure("Figure 3")
draw_digits(mus[1], list(range(1, allK[1]+1)))
plt.figure("Figure 4")
draw_digits(mus[2], list(range(1, allK[2]+1)))
plt.figure("Figure 5")
draw_digits(mus[3], list(range(1, allK[3]+1)))
plt.figure("Figure 6")
draw_digits(mus[4], list(range(1, allK[4]+1)))